In [1]:
import os 

In [2]:
os.chdir('/home/yehudan/Desktop/DecisionTree/DecisionTree')

In [1]:
import time
from datasets import mse_comparison 
from scipy.io.arff import loadarff
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [4]:
df = pd.read_csv('datasets/Pakistan_Largest_Ecommerce_Dataset.csv')

/data/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,3,7,8,9,11,12,13,14,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
def move_columns_to_end(df,col1,col2=None):
    cols = list(df.columns.values) #Make a list of all of the columns in the df
    cols.pop(cols.index(col1)) #Remove col1 from list
    if col2:
        cols.pop(cols.index(col2)) #Remove col2 from list
    if col2:
        df = df[cols+[col1,col2]] #Create new dataframe with columns in the order you want
    else:
        df = df[cols+[col1]]
    return df

In [6]:
def preprocessing(df):
    df1= df.copy()
    df1['Mobiles & Tablets'] = df['Mobiles & Tablets'].apply(lambda x: 0 if x<=df['Mobiles & Tablets'].median() else 1)
    df1['Beauty & Grooming'] = df['Beauty & Grooming'].apply(lambda x: 0 if x<=df['Beauty & Grooming'].median() else 1)
    return df1

In [7]:
pakistan_columns = ['grand_total', 'category_name_1', 'payment_method', 'Customer ID']
pakistan_filtered = df[pakistan_columns]
grouped_data = pakistan_filtered.groupby(by=['category_name_1', 'Customer ID'], as_index=False).agg({'grand_total': 'sum'})
pivoted = grouped_data.pivot(index='Customer ID', columns='category_name_1', values='grand_total')
pivoted = pivoted.reset_index().rename_axis(None, axis=1)
# pivoted = pivoted.fillna(0)
# final_df = pivoted.drop(['Customer ID'], axis='columns')
# final_df.head(2)

In [9]:
pivoted[pivoted.notnull().sum(axis=1)>=9].shape

(1325, 17)

In [10]:
final_df = pivoted[pivoted.notnull().sum(axis=1)>=9]
final_df = final_df.drop(['Customer ID'], axis='columns')
final_df = final_df.fillna(0)
final_df.shape

(1325, 16)

In [13]:
final_df = move_columns_to_end(final_df,'Mobiles & Tablets','Beauty & Grooming')
final_df.head()

,Appliances,Books,Computing,Entertainment,Health & Sports,Home & Living,Kids & Baby,Men's Fashion,Others,School & Education,Soghaat,Superstore,Women's Fashion,\N,Mobiles & Tablets,Beauty & Grooming
3,285592.7,0.0,99.00,593501.20,999.0,22185.0,2250.0,97804.55,1249.00,495.0,32990.0,94530.0,29158.50,115044.85,1257478.35,34219.30
12,207775.3,0.0,3230.00,53644.00,0.0,130560.0,3018.0,27770.00,0.00,720.0,38498.0,0.0,41868.00,12974.00,30886.00,66022.00
14,0.0,1730.0,1205.00,0.00,0.0,12171.0,1320.0,4721.00,125971.93,0.0,1065.0,2234.0,0.00,0.00,1205.00,0.00
15,0.0,6399.0,28345.11,0.00,3041.0,1475.0,15247.0,33108.60,4630.00,1863.0,9127.0,45546.0,1927773.13,994.00,185968.40,54005.22
19,1363251.0,0.0,360.00,1076207.68,13536.2,10230.0,1740.0,110845.40,0.00,0.0,21671.5,328188.0,48709.00,124325.25,1219169.35,37893.00


In [14]:
final_df_classifiction = preprocessing(final_df)
final_df_classifiction.head()

,Appliances,Books,Computing,Entertainment,Health & Sports,Home & Living,Kids & Baby,Men's Fashion,Others,School & Education,Soghaat,Superstore,Women's Fashion,\N,Mobiles & Tablets,Beauty & Grooming
3,285592.7,0.0,99.00,593501.20,999.0,22185.0,2250.0,97804.55,1249.00,495.0,32990.0,94530.0,29158.50,115044.85,1,1
12,207775.3,0.0,3230.00,53644.00,0.0,130560.0,3018.0,27770.00,0.00,720.0,38498.0,0.0,41868.00,12974.00,1,1
14,0.0,1730.0,1205.00,0.00,0.0,12171.0,1320.0,4721.00,125971.93,0.0,1065.0,2234.0,0.00,0.00,0,0
15,0.0,6399.0,28345.11,0.00,3041.0,1475.0,15247.0,33108.60,4630.00,1863.0,9127.0,45546.0,1927773.13,994.00,1,1
19,1363251.0,0.0,360.00,1076207.68,13536.2,10230.0,1740.0,110845.40,0.00,0.0,21671.5,328188.0,48709.00,124325.25,1,1


In [15]:
final_df_classifiction['Beauty & Grooming'].value_counts()

0    663
1    662
Name: Beauty & Grooming, dtype: int64

1325 rows (more than 9 not nulls)

In [13]:
_, _, _, mse_choose = mse_comparison.mse_comparison(path=None,
                                                                    df=final_df_classifiction, n_features=14,
                                                                    max_depth=np.inf,
                                                           min_samples_leaf=int(0.05*final_df_classifiction.shape[0]),
                                                                    n_splits=10, model='gb', n_estimators=50,only_choose_one=True,
                                                                    regression=False,
                                                                    random_state=0)

Cell execution time: 2023-04-27 06:45:22.920839


CV progress bar: 0it [00:00, ?it/s]

starting choose one model



Choose one progress bar: 0it [00:00, ?it/s]

Starting all
runtime for all model:41.68399500846863 seconds
Finish all
Starting ind


...


Choose one progress bar: 10it [21:50, 131.00s/it]

runtime for ind model:89.60092377662659 seconds
Finish ind
MSE all: 3.1771358543417367
MSE ind: 0
for ind was chosen



CV progress bar: 10it [3:56:11, 1417.14s/it]

runtime for ind model:104.85729026794434 seconds
runtime for choose one model:1414.874113559723 seconds
MSE for all model:   nan
MSE for ind model:   nan
MSE LOO model:       nan
MSE choose one model:0.31962576896787426
stdev choose one model:0.03929150973271142
Total runtime: 236.18944395780562 minutes



/home/yehudan/Desktop/DecisionTree/DecisionTree/datasets/mse_comparison.py:276: RuntimeWarning: Mean of empty slice.
  print(f'MSE for all model:   {np.array(mse_all).mean()}')
/data/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/yehudan/Desktop/DecisionTree/DecisionTree/datasets/mse_comparison.py:277: RuntimeWarning: Mean of empty slice.
  print(f'MSE for ind model:   {np.array(mse_ind).mean()}')
/home/yehudan/Desktop/DecisionTree/DecisionTree/datasets/mse_comparison.py:278: RuntimeWarning: Mean of empty slice.
  print(f'MSE LOO model:       {np.array(mse_loo).mean()}')


In [15]:
res_Kfold_all, res_Kfold_ind, res_Kfold_loo, res_Kfold_choose  = mse_comparison.mse_comparison(path=None,
                                                                    df=final_df_classifiction, n_features=14,
                                                                    max_depth=np.inf,
                                                           min_samples_leaf=int(0.05*final_df_classifiction.shape[0]),
                                                                    n_splits=10, model='gb', n_estimators=50,
                                                                    regression=False,
                                                                    random_state=0)

Cell execution time: 2023-04-30 09:10:33.976262


CV progress bar: 0it [00:00, ?it/s]

for all print:
runtime for all model:55.160425424575806 seconds
for ind print:
print for target number: 0
print for target number: 1
runtime for ind model:109.94054555892944 seconds
LOO print:


CV progress bar: 1it [30:21, 1821.90s/it]

runtime for LOO model:1656.7920179367065 seconds
Score for all:0.32706766917293234
Score for ind:0.33458646616541354
Score for loo model:0.3007518796992481
****************************************************************************************************************************************************************
for all print:
runtime for all model:52.30332851409912 seconds
for ind print:
print for target number: 0
print for target number: 1
runtime for ind model:112.7987654209137 seconds
LOO print:


...

runtime for LOO model:1659.9165861606598 seconds
Score for all:0.38257575757575757
Score for ind:0.38636363636363635
Score for loo model:0.35984848484848486
****************************************************************************************************************************************************************
for all print:
runtime for all model:57.44198417663574 seconds
for ind print:
print for target number: 0
print for target number: 1
runtime for ind model:121.45532703399658 seconds
LOO print:


CV progress bar: 9it [4:29:37, 1813.78s/it]

runtime for LOO model:1663.3147764205933 seconds
Score for all:0.2727272727272727
Score for ind:0.2727272727272727
Score for loo model:0.2689393939393939
****************************************************************************************************************************************************************
for all print:
runtime for all model:52.59410262107849 seconds
for ind print:
print for target number: 0
print for target number: 1
runtime for ind model:111.14598751068115 seconds
LOO print:


CV progress bar: 10it [4:58:41, 1792.12s/it]

runtime for LOO model:1580.3379695415497 seconds
Score for all:0.3446969696969697
Score for ind:0.3446969696969697
Score for loo model:0.32575757575757575
****************************************************************************************************************************************************************
MSE for all model:   0.32229152426520846
MSE for ind model:   0.31962576896787426
MSE LOO model:       0.31323763955342904
Total runtime: 298.6875813126564 minutes


In [16]:
res_Kfold_all, res_Kfold_ind, res_Kfold_loo, res_Kfold_choose  = mse_comparison.mse_comparison(path=None,
                                                                    df=final_df_classifiction, n_features=14,
                                                                    max_depth=np.inf,
                                                           min_samples_leaf=int(0.05*final_df_classifiction.shape[0]),
                                                                    n_splits=10, model='gb', n_estimators=50,
                                                                    regression=False,
                                                                    random_state=0)

Cell execution time: 2023-05-03 09:15:40.955472


CV progress bar: 0it [00:00, ?it/s]

for all print:
runtime for all model:49.1641206741333 seconds
for ind print:
print for target number: 0
print for target number: 1
runtime for ind model:105.81240582466125 seconds
LOO print:


CV progress bar: 1it [28:43, 1723.26s/it]

runtime for LOO model:1568.282131910324 seconds
Score for all:0.32706766917293234
Score for ind:0.33458646616541354
Score for loo model:0.3007518796992481
****************************************************************************************************************************************************************
for all print:
runtime for all model:49.75111484527588 seconds
for ind print:
print for target number: 0
print for target number: 1
runtime for ind model:105.30720043182373 seconds
LOO print:


...

runtime for LOO model:1575.7219092845917 seconds
Score for all:0.38257575757575757
Score for ind:0.38636363636363635
Score for loo model:0.35984848484848486
****************************************************************************************************************************************************************
for all print:
runtime for all model:50.757460594177246 seconds
for ind print:
print for target number: 0
print for target number: 1
runtime for ind model:108.12784814834595 seconds
LOO print:


CV progress bar: 9it [4:17:17, 1730.27s/it]

runtime for LOO model:1598.2587015628815 seconds
Score for all:0.2727272727272727
Score for ind:0.2727272727272727
Score for loo model:0.2689393939393939
****************************************************************************************************************************************************************
for all print:
runtime for all model:49.4195499420166 seconds
for ind print:
print for target number: 0
print for target number: 1
runtime for ind model:106.77682995796204 seconds
LOO print:


CV progress bar: 10it [4:45:07, 1710.75s/it]

runtime for LOO model:1513.479932308197 seconds
Score for all:0.3446969696969697
Score for ind:0.3446969696969697
Score for loo model:0.32575757575757575
****************************************************************************************************************************************************************
MSE for all model:   0.32229152426520846
MSE for ind model:   0.31962576896787426
MSE LOO model:       0.31323763955342904
Total runtime: 285.1257231235504 minutes


In [17]:
import statistics
print(f'for all stdev {statistics.stdev(res_Kfold_all)}')
print(f'for ind stdev {statistics.stdev(res_Kfold_ind)}')
print(f'for loo stdev {statistics.stdev(res_Kfold_loo)}')

for all stdev 0.037863202717161905
for ind stdev 0.03929150973271142
for loo stdev 0.02810874515516123
